In [17]:
sc.version

u'1.6.0'

In [18]:
def set_hadoop_config(credentials):
    prefix = "fs.swift.service." + credentials['name'] 
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + ".auth.url", credentials['auth_url']+'/v3/auth/tokens')
    hconf.set(prefix + ".auth.endpoint.prefix", "endpoints")
    hconf.set(prefix + ".tenant", credentials['project_id'])
    hconf.set(prefix + ".username", credentials['user_id'])
    hconf.set(prefix + ".password", credentials['password'])
    hconf.setInt(prefix + ".http.port", 8080)
    hconf.set(prefix + ".region", credentials['region'])
    hconf.setBoolean(prefix + ".public", True)

In [ ]:
#Download the csv file from
# http://spatialkeydocs.s3.amazonaws.com/FL_insurance_sample.csv.zip
# Unzip the file and then drop CSV in the Data Sources area.

In [ ]:
#Drop your file into Object storage and then when you click insert to code you will see below kind of credential 
#dictionary object added.

In [19]:
objectStorageCreds = {
  'auth_uri':'',
  'global_account_auth_uri':'',
  'username':'Admin_5b8faff871d87700d342130e7cc52e06a7019feb',
  'password':"""XXXXXXXXX""",
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_216c032f_3f57_4763_ae97_5c6a83a0d523',
  'project_id':'e097bbd898534ed1ad0e45c82baedb2d',
  'region':'dallas',
  'user_id':'a493676500794827b020874099c5ee1c',
  'domain_id':'da5b6dd1c8374f67b1050172badbef8c',
  'domain_name':'837523',
  'container':'1464138585443',
  'tenantId':'sf58-a0340cf1eaefe8-2c631c8ff999',
  'filename':'FL_insurance_sample.csv'
}

In [ ]:
#Name the credentials anything that you want, just don't use spark word in there.

In [20]:
objectStorageCreds['name'] = 'testcredname'

In [21]:
set_hadoop_config(objectStorageCreds)

In [22]:
w_data = sc.textFile("swift://" + objectStorageCreds['container'] + "." + objectStorageCreds['name'] + "/" + objectStorageCreds['filename'])

In [23]:
print w_data.count()

36635


In [26]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [27]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load("swift://" + objectStorageCreds['container'] + "." + objectStorageCreds['name'] + "/" + objectStorageCreds['filename'])

In [28]:
df.printSchema()

root
 |-- policyID: integer (nullable = true)
 |-- statecode: string (nullable = true)
 |-- county: string (nullable = true)
 |-- eq_site_limit: double (nullable = true)
 |-- hu_site_limit: double (nullable = true)
 |-- fl_site_limit: double (nullable = true)
 |-- fr_site_limit: double (nullable = true)
 |-- tiv_2011: double (nullable = true)
 |-- tiv_2012: double (nullable = true)
 |-- eq_site_deductible: double (nullable = true)
 |-- hu_site_deductible: double (nullable = true)
 |-- fl_site_deductible: double (nullable = true)
 |-- fr_site_deductible: integer (nullable = true)
 |-- point_latitude: double (nullable = true)
 |-- point_longitude: double (nullable = true)
 |-- line: string (nullable = true)
 |-- construction: string (nullable = true)
 |-- point_granularity: integer (nullable = true)



In [29]:
outputfilename="filteredResidential.csv"

In [35]:
df.filter(df['line']=='Residential').write.save("swift://" + objectStorageCreds['container'] + "." + objectStorageCreds['name'] + "/" + outputfilename)